In [1]:
import carball
from carball.json_parser.game import Game
from carball.analysis.analysis_manager import AnalysisManager

In [4]:
def col_names(df):
    blue_name = df.columns[1][0]
    orange_name = df.columns[-1][0]
  
  
    a = []
    for i in df.columns:
        if i[0] == blue_name:
            a.append("blue_" + i[1])
        elif i[0] == orange_name:
            a.append("orange_" + i[1])
        else:
            a.append("_".join(i))

    df.columns = a
    return df

In [5]:
def prepare(file):
    _json = carball.decompile_replay(file)
    game = Game()
    game.initialize(loaded_json=_json)
    
    analysis_manager = AnalysisManager(game)
    analysis_manager.create_analysis()
    
    dataframe = analysis_manager.get_data_frame()
    dataframe = col_names(dataframe)
    
    if dataframe.shape[1] != 66:
        dataframe.insert(61, 'game_is_overtime', 'no_OT')
    
    dataframe.insert(0, "ID", dataframe.index)
    
    return dataframe

In [6]:
def next_goal_regulation(row, df):
    try:
        # find frame before next goal
        ng = df.loc[(df["game_goal_number"].isnull()) & (df["ID"] > row), "ID"].iloc[0] - 1
        
        # depending on the ball location, we can tell who scored the next goal
        if df.loc[ng, "ball_pos_y"] > 0:
            return 0
        else:
            return 1
    except:
        return None

    
def next_goal_ot(df):
    # check where the ball is to end overtime
    if df.iloc[-1, df.columns.get_loc("ball_pos_y")] > 0:
        # if ball is on orange half, team 0 (blue) scored in ot
        return 0
    else:
        # else team 1 (orange) scored
        return 1


def next_goal(df):
    if df.loc[1, "game_is_overtime"] == "no_OT":
        # if game doesn't go to overtime, we just need to run the regulation function
        df["game_next_goal"] = df["ID"].apply(lambda x: next_goal_regulation(x, df))
    else:
        # if game goes to overtime, split df into ot and regulation
        df_ot = df.loc[df["game_is_overtime"].isnull() == False, ]
        df = df.loc[df["game_is_overtime"].isnull(), ]
        
        # run ot function on the ot dataframe
        df_ot["game_next_goal"] = next_goal_ot(df_ot)
        
        # run regulation function on the regulation dataframe
        df["game_next_goal"] = df["ID"].apply(lambda x: next_goal_regulation(x, df))
        
        # bind the dataframe back together
        df = df.append(df_ot)
    return df
        


In [36]:
# non overtime replay
# replay = "./Replays/practice/5c402249-2714-47a3-bd98-48d7e631e95d.replay"

# overtime replay
replay = "./Replays/practice/924303e9-4a99-4ef8-9c15-13484b7bf72f.replay"

df = prepare(replay)
df.shape

Goal is not shot: frame 5292 by Dider
Goal is not shot: frame 9079 by fivra
The player never hit the ball during the "carry"
The player never hit the ball during the "carry"


(11693, 67)

In [37]:
# find next goal scoring team at each frame
df = next_goal(df)
df.shape

C:\Users\mhugh\AppData\Local\Temp/ipykernel_6280/3185757653.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ot["game_next_goal"] = next_goal_ot(df_ot)
C:\Users\mhugh\AppData\Local\Temp/ipykernel_6280/3185757653.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["game_next_goal"] = df["ID"].apply(lambda x: next_goal_regulation(x, df))


(11693, 68)

In [38]:
# remove all columns of instances after goals
df = df.loc[df["game_goal_number"].isnull() == False, ]
df.head()

,ID,blue_ping,blue_pos_x,blue_pos_y,blue_pos_z,blue_vel_x,blue_vel_y,blue_vel_z,blue_ang_vel_x,blue_ang_vel_y,...,game_time,game_delta,game_seconds_remaining,game_replicated_seconds_remaining,game_is_overtime,game_ball_has_been_hit,game_goal_number,blue_boost_collect,orange_boost_collect,game_next_goal
111,111,6,2047.640015,2559.600098,17.01,-282.199993,-321.699982,0.8,-0.5,0.1,...,7.671802,0.041256,300.0,0.0,None,None,0.0,NaN,NaN,0.0
112,112,6,2047.640015,2559.600098,17.01,-282.199993,-321.699982,0.8,-0.5,0.1,...,7.707411,0.035609,300.0,0.0,None,None,0.0,NaN,NaN,0.0
113,113,6,2039.140015,2549.770020,17.01,-1601.900024,-1867.899933,0.0,-0.2,-0.1,...,7.745100,0.037688,300.0,0.0,None,None,0.0,NaN,NaN,0.0
114,114,6,2039.140015,2549.770020,17.01,-1601.900024,-1867.899933,0.0,-0.2,-0.1,...,7.781739,0.036640,300.0,0.0,None,None,0.0,NaN,NaN,0.0
115,115,6,2024.660034,2533.050049,17.01,-2626.199951,-2982.200012,0.0,1.3,0.8,...,7.819203,0.037464,300.0,0.0,None,None,0.0,NaN,NaN,0.0


In [24]:
df.loc[453, "game_seconds_remaining"]

290.0

In [27]:
df[["game_seconds_remaining", "game_next_goal"]].sample(1)

,game_seconds_remaining,game_next_goal
2171,237.0,0.0


In [7]:
import os

In [8]:
directory = os.fsencode("./Replays/test")
i = 0
for file in os.listdir(directory):
    df = prepare("./Replays/test/" + os.fsdecode(file))
    df = next_goal(df)
    if i == 0:
        df_total = df
        i += 1
    else:
        df_total = df_total.append(df)

Found bot not in bot list
Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 1640 by Stormiity
Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 775 by Fahad
Goal is not shot: frame 2193 by Ash.☆
Goal is not shot: frame 3981 by Fahad
The player never hit the ball during the "carry"
C:\Users\mhugh\anaconda3\envs\carball\lib\site-packages\numpy\lib\format.py:431: UserWarning: Stored array in format 3.0. It can only be read by NumPy >= 1.17
  header = _wrap_header_guess_version(header)
Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 2085 by crossbar
Goal is not shot: frame 3863 by Lubby
Goal is not shot: frame 6269 by crossbar
The player never hit the ball during the "carry"
The player never hit the ball during the "carry"
Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 6291 by zord
The player never hit the ball during the "carry"
Dropping these columns[('game', 'is_overtime')]
Goal is not shot: fra

In [9]:
df_total.shape

(289968, 68)

In [12]:
df_total.to_csv("testing_data.csv")

In [17]:
directory = os.fsencode("./Replays/train1")
i = 0
for file in os.listdir(directory):
    df = prepare("./Replays/train1/" + os.fsdecode(file))
    df = next_goal(df)
    if i == 0:
        df_total_train = df
        i += 1
    else:
        df_total_train = df_total.append(df)

Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 3886 by Nuqqet
Goal is not shot: frame 17570 by smashy
The player never hit the ball during the "carry"
The player never hit the ball during the "carry"
Goal is not shot: frame 527 by Kito
Goal is not shot: frame 4994 by Kito
The player never hit the ball during the "carry"
The player never hit the ball during the "carry"
C:\Users\mhugh\AppData\Local\Temp/ipykernel_2448/3185757653.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ot["game_next_goal"] = next_goal_ot(df_ot)
C:\Users\mhugh\AppData\Local\Temp/ipykernel_2448/3185757653.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = 

In [18]:
df_total_train.shape

(296413, 68)

In [19]:
df_total_train.to_csv("training_data1.csv")

In [20]:
directory = os.fsencode("./Replays/train2")
i = 0
for file in os.listdir(directory):
    df = prepare("./Replays/train2/" + os.fsdecode(file))
    df = next_goal(df)
    if i == 0:
        df_total_train = df
        i += 1
    else:
        df_total_train = df_total.append(df)

Dropping these columns[('game', 'is_overtime')]
The player never hit the ball during the "carry"
Dropping these columns[('game', 'is_overtime')]
Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 4383 by nakeiz.
Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 9612 by Wilfred
The player never hit the ball during the "carry"
The player never hit the ball during the "carry"
C:\Users\mhugh\AppData\Local\Temp/ipykernel_2448/3185757653.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ot["game_next_goal"] = next_goal_ot(df_ot)
C:\Users\mhugh\AppData\Local\Temp/ipykernel_2448/3185757653.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [21]:
df_total_train.shape

(302825, 68)

In [22]:
df_total_train.to_csv("training_data2.csv")

In [23]:
directory = os.fsencode("./Replays/train3")
i = 0
for file in os.listdir(directory):
    df = prepare("./Replays/train3/" + os.fsdecode(file))
    df = next_goal(df)
    if i == 0:
        df_total_train = df
        i += 1
    else:
        df_total_train = df_total.append(df)

Goal is not shot: frame 2956 by smashy
Goal is not shot: frame 12505 by smashy
C:\Users\mhugh\AppData\Local\Temp/ipykernel_2448/3185757653.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ot["game_next_goal"] = next_goal_ot(df_ot)
C:\Users\mhugh\AppData\Local\Temp/ipykernel_2448/3185757653.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["game_next_goal"] = df["ID"].apply(lambda x: next_goal_regulation(x, df))
Dropping these columns[('game', 'is_overtime')]
Dropping these columns[('game', 'is_

In [24]:
df_total_train.shape

(296850, 68)

In [25]:
df_total_train.to_csv("training_data3.csv")

In [26]:
directory = os.fsencode("./Replays/train4")
i = 0
for file in os.listdir(directory):
    df = prepare("./Replays/train4/" + os.fsdecode(file))
    df = next_goal(df)
    if i == 0:
        df_total_train = df
        i += 1
    else:
        df_total_train = df_total.append(df)

Found bot not in bot list
Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 6171 by Bweary!
The player never hit the ball during the "carry"
The player never hit the ball during the "carry"
Dropping these columns[('game', 'is_overtime')]
Goal is not shot: frame 4744 by sky
Goal is not shot: frame 40390 by dripsplash
The player never hit the ball during the "carry"
C:\Users\mhugh\AppData\Local\Temp/ipykernel_2448/3185757653.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ot["game_next_goal"] = next_goal_ot(df_ot)
C:\Users\mhugh\AppData\Local\Temp/ipykernel_2448/3185757653.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value i

In [27]:
df_total_train.shape

(342097, 68)

In [28]:
df_total_train.to_csv("training_data4.csv")